In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Defensive Linemen Number of Pressures

In [2]:
pass_rush_summary = pd.read_csv('../data/pass_rush_summary.csv')

In [5]:
grouped_DL_pressures = pass_rush_summary.groupby("team_name").agg({
    "true_pass_set_total_pressures": "sum",
}).reset_index()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(grouped_DL_pressures[['true_pass_set_total_pressures']])

grouped_DL_pressures_scaled = pd.DataFrame(
    scaled_data, 
    columns=["true_pass_set_total_pressures"]
)

grouped_DL_pressures_scaled["team_name"] = grouped_DL_pressures["team_name"]

grouped_DL_pressures_scaled = grouped_DL_pressures_scaled.sort_values(
    by="true_pass_set_total_pressures", ascending=False
)


In [6]:
grouped_DL_pressures_scaled

,true_pass_set_total_pressures,team_name
25,2.075930,PHI
10,1.762012,DET
9,1.665422,DEN
16,1.496390,LA
15,1.303210,KC
2,0.844408,BLT
3,0.506343,BUF
29,0.506343,TB
26,0.458048,PIT
31,0.458048,WAS


# Cornerback Number of Targets

In [9]:
cb_summary = pd.read_csv('../data/defense_summary.csv')

In [11]:
from sklearn.preprocessing import StandardScaler

higher_is_better = [
    "catch_rate", "grades_defense", "grades_defense_penalty", 
    "grades_pass_rush_defense", "interceptions", "tackles", 
    "tackles_for_loss", "total_pressures"
]

lower_is_better = ["qb_rating_against", "declined_penalties", "targets"]

cb_metrics = higher_is_better + lower_is_better

grouped_cb = cb_summary.groupby("team_name")[cb_metrics].sum().reset_index()

numeric_columns = grouped_cb.select_dtypes(include=["number"]).columns

# Standardizing the numeric columns
scaler = StandardScaler()
scaled_data = scaler.fit_transform(grouped_cb[numeric_columns])

grouped_cb_scaled = pd.DataFrame(scaled_data, columns=numeric_columns)
grouped_cb_scaled["team_name"] = grouped_cb["team_name"]

# Reversing the direction for metrics where lower is better
for col in lower_is_better:
    grouped_cb_scaled[col] *= -1  

weights = {
    "catch_rate": 1.2,
    "grades_defense": 1.0,
    "grades_defense_penalty": 0.8,
    "grades_pass_rush_defense": 1.1,
    "interceptions": 1.3,
    "tackles": 1.0,
    "tackles_for_loss": 1.2,
    "total_pressures": 1.5,
    "qb_rating_against": 1.4,  
    "declined_penalties": 1.0,  
    "targets": 0.9  
}

grouped_cb_scaled["cb_safety_rating"] = sum(
    grouped_cb_scaled[col] * weight for col, weight in weights.items()
)

grouped_cb_scaled = grouped_cb_scaled.sort_values(by="cb_safety_rating", ascending=False)

grouped_cb_scaled = grouped_cb_scaled.reset_index(drop=True)


In [13]:
grouped_cb_scaled[['team_name', "targets"]].sort_values(by="targets", ascending=False)

,team_name,targets
21,TEN,1.435605
20,CHI,1.347124
4,NYG,1.347124
29,NE,1.294036
6,CLV,1.170162
14,CAR,1.152466
30,DAL,0.975504
19,NYJ,0.922415
22,SF,0.727757
24,SEA,0.710061
